In [1]:
import pandas as pd

In [2]:
path_data = 'C:/Users/ISHMO_CT/Downloads/my-projects/NLP_env/NLP_Disaster_Tweets_Prediction/nlp-getting-started/train.csv'

In [3]:
df = pd.read_csv(path_data)

In [4]:
df.head()

,id,keyword,location,text,target
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1


In [5]:
# Tokenise the text  load the model that cleans and tokenises the text
import spacy
nlp = spacy.load('en_core_web_sm')

In [7]:
import re
def tokenise(doc):
    doc = doc.lower()
    # remove punctuations 
    doc = re.sub(r'[^\w\s]', '', doc)
    # remove numbers 
    doc = re.sub(r'\d+', '', doc)
    # remove extra space 
    doc = doc.strip()
    tokens = [token.text for token in nlp(doc) if not token.is_punct and not token.is_stop]
    return tokens

df['tokens'] = df['text'].apply(lambda x: tokenise(x))

In [8]:
df.head()

,id,keyword,location,text,target,tokens
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1,"[deeds, reason, earthquake, allah, forgive]"
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1,"[forest, fire, near, la, ronge, sask, canada]"
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1,"[residents, asked, shelter, place, notified, o..."
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1,"[people, receive, wildfires, evacuation, order..."
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1,"[got, sent, photo, ruby, alaska, smoke, wildfi..."


In [12]:
# turn the tokens into vectors 
from sklearn.feature_extraction.text import TfidfVectorizer
# instatiate the class 
tfidfvectorizer = TfidfVectorizer()
dtm = tfidfvectorizer.fit_transform(df['tokens'].apply(lambda x : ' '.join(x)))

feature_names = tfidfvectorizer.get_feature_names_out()
dtm_df = pd.DataFrame(dtm.toarray(), columns=feature_names)
print(dtm_df)

      a__cee   aa  aaaa  aaaaaaallll  aaaaaand  aaarrrgghhh  aace_org  aal  \
0        0.0  0.0   0.0          0.0       0.0          0.0       0.0  0.0   
1        0.0  0.0   0.0          0.0       0.0          0.0       0.0  0.0   
2        0.0  0.0   0.0          0.0       0.0          0.0       0.0  0.0   
3        0.0  0.0   0.0          0.0       0.0          0.0       0.0  0.0   
4        0.0  0.0   0.0          0.0       0.0          0.0       0.0  0.0   
...      ...  ...   ...          ...       ...          ...       ...  ...   
7608     0.0  0.0   0.0          0.0       0.0          0.0       0.0  0.0   
7609     0.0  0.0   0.0          0.0       0.0          0.0       0.0  0.0   
7610     0.0  0.0   0.0          0.0       0.0          0.0       0.0  0.0   
7611     0.0  0.0   0.0          0.0       0.0          0.0       0.0  0.0   
7612     0.0  0.0   0.0          0.0       0.0          0.0       0.0  0.0   

      aampb  aampw  ...  ûïwhen  ûïymcglaun  ûïyou   ûò  ûòthe 

In [14]:
# we are going to compare the text with a statemne tthat contains more  
statement_about_distater = """The devastating earthquake that struck the coastal city 
triggered a massive tsunami, causing widespread destruction and flooding, reminiscent of
 the catastrophic 2004 Indian Ocean tsunami. The disaster response team sprang into action, 
 conducting search and rescue operations amidst the rubble, while emergency evacuation 
 efforts were underway to relocate survivors to safer areas. Unfortunately, the city's 
 industrial sector suffered a chemical spill, exacerbating the environmental disaster. 
 As relief efforts continued, aid workers struggled to manage the humanitarian crisis, 
 providing trauma management and support to those affected. The tragedy was likened to 
 the Great Chicago Fire of 1871 and the San Francisco earthquake of 1906, with many 
 drawing parallels to the Hiroshima and Nagasaki bombings of 1945. Meanwhile, 
 investigators probed the catastrophic failure of the city's infrastructure, 
 pointing to a combination of human error and natural disaster. The aftermath of the 
 disaster saw a massive recovery effort, with workers toiling tirelessly to rebuild and 
 revitalize the community, still reeling from the shock of the plane crash that had occurred 
 just days prior. As the city began to heal, residents reflected on past disasters, including 
 the Holocaust, the Irish Potato Famine, and the Black Death, acknowledging the resilience of 
 the human spirit in the face of adversity. The disaster served as a stark reminder of the 
 importance of disaster preparedness and emergency planning, particularly in areas prone to 
 wildfires, landslides, and droughts. In the end, the community emerged stronger, 
 more united, and determined to mitigate the impact of future disasters, 
 whether natural or man-made, such as oil spills, nuclear meltdowns, or 
 industrial accidents."""

In [18]:
# we are using Nearestneighbour to compare out vctors with the paragraph above 
from sklearn.neighbors import NearestNeighbors
nn = NearestNeighbors(n_neighbors = 30)
# fit eith the dtm 
nn.fit(dtm)

NearestNeighbors(n_neighbors=30)

In [19]:
# find the most similar 30 words 
# vectorise the statement 
statement_about_distater_vectorised = tfidfvectorizer.transform([statement_about_distater])

# Query the 30 most simillar 
distance, indices = nn.kneighbors(statement_about_distater_vectorised)

# print the 30 most simillar texts in the data frame text column 
for idx in indices[0]:
    print(df.loc[idx, 'text'])

Some people are really natural disaster too
natural disaster ÛÒ News Stories About natural disaster - Page 1 | Newser http://t.co/TB8gZEMbXU
she's a natural disaster she's the last of the American girls ??
I'm setting myself up for disaster
What Natural Disaster Are You When You Get Angry? http://t.co/O9DzgZqEMf
What Natural Disaster Are You When You Get Angry? http://t.co/q4gl3Dvhu1
We want to see no more Hiroshima and Nagasaki nuclear bomb disaster in this beautiful world.Lets be peaceful &amp; save this human civilization.
At FEMA's Center for Disaster Preparedness for a weeklong training on Chemical Biological Radioactive Nuclear   Emergency Response
@ConnorFranta #AskConnor if you were a natural disaster what would you be?
Rise up like a natural disaster we take the bat and then we take back the town????
I'm a disaster?? https://t.co/VCV73BUaCZ
Rationing of food and water may also become necessary during an emergency such as a natural disaster or terror attack.
'Up to 40% of busi